In [1]:
import numpy as np
import pandas as pd
import iqplot
import bokeh.io
import csv
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def get_maxratios(files):
    """Generates a csv file of the solid to gas ratios with its associated parameters"""
    t_plot = [1e4,2e4,3e4,4e4,5e4,6e4,7e4,8e4,9e4,1e5]
    filename = 'max_ratio.csv'
    values = []
    head = np.array(['Time', 'Tau', 'Alpha', 'St', 'Dust/Gas', 'Pebble/Gas', '(Dust+Pebble)/Gas'])
    values.append(head)
    for idx, time in enumerate(t_plot):
        for file in files:
            df = pd.read_csv(file + '/disk_' + str(int(time)) + '.txt', delimiter=',', header=None)
            sigma_gas_plot = df[1].to_numpy().astype(np.float)
            cor_dust = df[2].to_numpy().astype(np.float)
            calc_dust = df[3].to_numpy().astype(np.float)
            for_dust = df[4].to_numpy().astype(np.float)
            cor_peb = df[8].to_numpy().astype(np.float)
            calc_peb = df[9].to_numpy().astype(np.float)
            for_peb = df[10].to_numpy().astype(np.float)
            sigma_dust_plot = cor_dust+calc_dust+for_dust
            sigma_peb_plot = cor_peb+calc_peb+for_peb
            pg_ratio = max(sigma_peb_plot/sigma_gas_plot)
            dg_ratio = max(sigma_dust_plot/sigma_gas_plot)
            pdg_ratio = max((sigma_peb_plot + sigma_dust_plot)/sigma_gas_plot)
            if('MRI' in file):
                alpha_label = 'alpha MRI'
            else:
                alpha_label = 'alpha 1e-3'
            if('Stx10' in file):
                St_label = 'St x 10'
            else:
                St_label = 'St 5e-2'
            tau_label = int(float(file[3:6]))
            output = np.array([int(time), tau_label, alpha_label, St_label, dg_ratio, pg_ratio, pdg_ratio], dtype=object)
#             print(output)
#             output = output.reshape(1, output.shape[0])
            values.append(output)
    values = np.array(values)
    with open(filename, "w") as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerows(values)

In [3]:
get_maxratios(['tau1e3_St5e-2_alpha1e-3', 'tau1e3_St5e-2_alphaMRI', 'tau1e4_St5e-2_alphaMRI','tau1e5_Stx10_alphaMRI','tau1e5_St5e-2_alphaMRI'])

In [4]:
def plot_maxratio(param):
    """Produces a strip of the solid to gas ratio against a categorical variable."""
    tooltips_lst ={"Tau" : [("Alpha","@Alpha"),("Stokes Number", "@St"),("Year","@Time")],
                   "Alpha" : [("Tau","@Tau"),("Stokes Number", "@St"),("Year","@Time")],
                   "St"  : [("Alpha","@Alpha"),("Tau", "@Tau"),("Year","@Time")],
                   "Time" : [("Alpha","@Alpha"),("Stokes Number", "@St"),("Tau","@Tau")]
                  }
    if(param not in tooltips_lst.keys()):
        raise KeyError("Inputted Parameter is not valid. " + 
                       "Expected one of 'Tau', 'Alpha', 'St', or 'Time'.")
    df = pd.read_csv("max_ratio.csv")
    dg = iqplot.strip(
        data=df, 
        q="Dust/Gas", 
        cats=param, 
        y_axis_label=param,
        title="Streaming Instability Conditions",
        tooltips=tooltips_lst[param]
    )
    pg = iqplot.strip(
        data=df, 
        q="Pebble/Gas", 
        cats="Tau", 
        y_axis_label=param,
        title="Streaming Instability Conditions",
        tooltips=tooltips_lst[param]
    )
    pdg = iqplot.strip(
        data=df, 
        q="(Dust+Pebble)/Gas", 
        cats="Tau", 
        y_axis_label=param,
        title="Streaming Instability Conditions",
        tooltips=tooltips_lst[param]
    )
    bokeh.io.show(bokeh.layouts.gridplot([dg,pg,pdg], ncols=1))
    

In [7]:
plot_maxratio('Alpha')